#Phase 2: Modeling & Hyperparameter Tuning 

In [ ]:
#from sklearn.model_selection import GridSearchCV
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt #graphs 
import tensorflow as tf 
#import sklearn as skl

**Train, Test, Validation**

In [ ]:
#Split validation set by 20%
#Image generator helps creating image aumentation to increase the amount of data we have
#Will implement various rotations and flips to the images to distort
train_gen = ImageDataGenerator(rescale = 1./255, validation_split=0.20)
test_gen = ImageDataGenerator(rescale = 1./255)


def get_train_set(train_gen, input_size):
  train_set = train_gen.flow_from_directory("/content/drive/MyDrive/DS 2 dataset/train",
                                         class_mode='categorical',
                                         target_size= input_size,
                                         color_mode = 'grayscale',
                                         batch_size= 32, 
                                         shuffle = True, 
                                         subset ='training')
  return train_set

def get_validation_set(train_gen, input_size):
  validation_set = train_gen.flow_from_directory("/content/drive/MyDrive/DS 2 dataset/train",
                                          target_size= input_size,
                                          color_mode = 'grayscale',
                                          class_mode='categorical',
                                          batch_size= 32, 
                                          shuffle = True, 
                                          subset ='validation')
  return validation_set

def get_test_set(test_gen, input_size):
  test_set = test_gen.flow_from_directory("/content/drive/MyDrive/DS 2 dataset/test",
                                         target_size=input_size, 
                                         color_mode = 'grayscale',
                                         class_mode='categorical',
                                         batch_size= 1,
                                         shuffle = True)
  return test_set

#Modeling & Hyper-parameter Tuning

In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 96 kB 4.7 MB/s 


In [ ]:
from tensorflow.keras.applications import xception #most parameters (44 mil)
import keras_tuner as kt
from kerastuner.tuners import Hyperband

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
  #input, conv2d, pooling,flatten ,Dense, output
  #Sequential = order layers are vistied
  #flatten layer = get vector to put in classifier 

  # 3 conv layers followed by max pool 
  # 3rd conv layer followed by a dropout layer to prevent overfitting 
  # flatten, fully connected layers 

def build_scratch_model(hp):
    model = tf.keras.Sequential([tf.keras.layers.Conv2D(filters = hp.Choice('num_filters', 
                                                                          values = [32,64], 
                                                                          default = 64), kernel_size=(3,3),
                                                      activation = 'relu',input_shape = (224,224,1)),
                               
                               tf.keras.layers.MaxPool2D(3,3),

                               tf.keras.layers.Conv2D(filters = hp.Choice('num_filters', values  = [32,64], default = 64), kernel_size=(3,3),activation='relu'),
                               tf.keras.layers.MaxPool2D(3,3),

                               tf.keras.layers.Conv2D(filters = hp.Choice('num_filters',
                                                                          values  = [32,64],
                                                                          default = 64), kernel_size=(3,3),
                                                      activation='relu'),

                               tf.keras.layers.Dropout(hp.Float('dropout',
                                                                min_value=0.0,
                                                                max_value=0.1,
                                                                default=0.005,
                                                                step=0.01)),
                               tf.keras.layers.MaxPool2D(3,3),

                               tf.keras.layers.Flatten(),

                               tf.keras.layers.Dense(units=hp.Int('units', min_value = 32,
                                                                  max_value=512,
                                                                  step=32),
                                                     activation= 'relu'),
                               
                               tf.keras.layers.Dense(196, activation='softmax')])
  
    model.compile(optimizer= tf.keras.optimizers.Adam(hp.Choice('learning_rate',
                      values=[1e-3])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
  
    return model 

In [ ]:
tuner_1 = Hyperband(hypermodel=build_scratch_model, objective='val_accuracy', max_epochs= 100)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner_1.search_space_summary()

Search space summary
Default search space size: 4
num_filters (Choice)
{'default': 64, 'conditions': [], 'values': [32, 64], 'ordered': True}
dropout (Float)
{'default': 0.005, 'conditions': [], 'min_value': 0.0, 'max_value': 0.1, 'step': 0.01, 'sampling': None}
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001], 'ordered': True}


In [ ]:
tuner_1.search(get_train_set(train_gen=train_gen, input_size = (224,224)), epochs = 50, validation_data = get_validation_set(train_gen=train_gen, input_size= (224,224)), callbacks = [stop_early])

Trial 254 Complete [00h 04m 18s]
val_accuracy: 0.11230283975601196

Best val_accuracy So Far: 0.17981073260307312
Total elapsed time: 06h 48m 02s
INFO:tensorflow:Oracle triggered exit


In [ ]:
 tuner_1.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_filters: 32
dropout: 0.04
units: 416
learning_rate: 0.001
tuner/epochs: 34
tuner/initial_epoch: 12
tuner/bracket: 4
tuner/round: 3
tuner/trial_id: a373e1f878f5b55ce3505086e5564143
Score: 0.17981073260307312
Trial summary
Hyperparameters:
num_filters: 64
dropout: 0.02
units: 224
learning_rate: 0.001
tuner/epochs: 12
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.1526813954114914
Trial summary
Hyperparameters:
num_filters: 32
dropout: 0.08
units: 512
learning_rate: 0.001
tuner/epochs: 12
tuner/initial_epoch: 4
tuner/bracket: 4
tuner/round: 2
tuner/trial_id: 54c77531eef707fccaf899c180f7db15
Score: 0.14826498925685883
Trial summary
Hyperparameters:
num_filters: 32
dropout: 0.09
units: 352
learning_rate: 0.001
tuner/epochs: 12
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.14258675277233124
Trial summary
Hyp

**Epoch Optimization**

In [ ]:
# get best hp
best_hps = tuner_1.get_best_hyperparameters(num_trials = 1)[0]
#build best model 
model = tuner_1.hypermodel.build(best_hps)

#train best model to find optimal number of epochs

hist = model.fit(get_train_set(train_gen=train_gen, input_size = (224,224)),epochs = 100, validation_data = get_validation_set(train_gen=train_gen, input_size= (224,224)))


NameError: ignored